# **Image Classifier**


### **Exploratory Data Analysis (EDA)**

###  **Importing Libraries**

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import json
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
import os
import shutil

###  **Problem statement and data collection**

In [2]:
import requests
import zipfile
import io

url = "https://storage.googleapis.com/datascience-materials/dogs-vs-cats.zip"
response = requests.get(url)
response.raise_for_status()

with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    z.extractall("dogs-vs-cats")

In [6]:
import os
import shutil

images_folder = "../src/dogs-vs-cats/dogs-vs-cats/train"
source_folder = "../data/processed"
cats_folder = "../data/processed/cats"
dogs_folder = "../data/processed/dogs"

# Crear carpetas si no existen
os.makedirs(source_folder, exist_ok=True)
os.makedirs(cats_folder, exist_ok=True)
os.makedirs(dogs_folder, exist_ok=True)

# Contadores para limitar 100 por clase
cat_count = 0
dog_count = 0
max_images = 100  # máximo por clase

for filename in os.listdir(images_folder):
    file_path = os.path.join(images_folder, filename)
    if os.path.isfile(file_path):
        nombre = filename.lower()
        if nombre.startswith("cat") and cat_count < max_images:
            shutil.copy(file_path, os.path.join(cats_folder, filename))
            cat_count += 1
        elif nombre.startswith("dog") and dog_count < max_images:
            shutil.copy(file_path, os.path.join(dogs_folder, filename))
            dog_count += 1
        
        # Romper el loop si ya tenemos 100 de cada clase
        if cat_count >= max_images and dog_count >= max_images:
            break


In [7]:
from keras.preprocessing import image

2025-08-20 18:19:43.677072: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-20 18:19:43.696281: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-20 18:19:44.867019: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-20 18:19:47.708736: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

In [27]:
# Damos etiquetas a las imagenes (Dario)
def load_and_preprocess_images(data_dir, target_size=(224, 224)):
    images = []
    labels = []

    for label in os.listdir(data_dir):
        label_dir = os.path.join(data_dir, label)
        if os.path.isdir(label_dir):
            for filename in os.listdir(label_dir):
                img_path = os.path.join(label_dir, filename)
                try:
                    img = image.load_img(img_path, target_size=target_size)
                    img_array = image.img_to_array(img)
                    img_array /= 255.0  # Normalizar los valores de píxeles
                    images.append(img_array)
                    # Asigna la etiqueta 0 para "Cat" y 1 para "Dog"
                    if label == "cats":
                        labels.append(0)
                    elif label == "dogs":
                        labels.append(1)
                except Exception as e:
                    print(f"Error cargando la imagen {img_path}: {e}")

    return np.array(images), np.array(labels)


images, labels = load_and_preprocess_images("../data/processed")

Lo de arriba esta ok

In [29]:
X_train, X_test, Y_train, Y_test = train_test_split(images,labels, test_size=0.2, random_state=42)

In [ ]:
#ESTO NO VA O REVISAR QUE SIRVE

import pandas as pd
import shutil
from sklearn.model_selection import train_test_split
import os

source_dir = "../data/processed"

train_dir = "../data/train"
test_dir = "../data/test"

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

classes = ["cats", "dogs"]

for cls in classes:
    cls_path = os.path.join(source_dir, cls)
    files = os.listdir(cls_path)
    
    train_files, test_files = train_test_split(files, test_size=0.2, random_state=42)
    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(test_dir, cls), exist_ok=True)
    
    for f in train_files:
        shutil.copy(os.path.join(cls_path, f), os.path.join(train_dir, cls, f))
    
    for f in test_files:
        shutil.copy(os.path.join(cls_path, f), os.path.join(test_dir, cls, f))

In [30]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [33]:
train_datagen = ImageDataGenerator()
train_datagen_10_percent = ImageDataGenerator(rescale=1/255.)


train_data = train_datagen_10_percent.flow_from_directory(X_train, Y_train,
                                                         class_mode='categorical',
                                                         batch_size=32,
                                                         shuffle=True)

test_data = train_datagen_10_percent.flow_from_directory(X_test,Y_test,                                        
                                                         class_mode='categorical',
                                                         batch_size=32,
                                                         shuffle=True)

ValueError: listdir: embedded null character in path

In [17]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten


model = Sequential()
model.add(Conv2D(input_shape = (224,224,3), filters = 64, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 64,kernel_size = (3,3),padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size = (2,2),strides = (2,2)))
model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size = (2,2),strides = (2,2)))
model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size = (2,2),strides = (2,2)))
model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size = (2,2),strides = (2,2)))
model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = "relu"))
model.add(MaxPool2D(pool_size = (2,2),strides = (2,2)))

# Mover los datos por las capas densas
model.add(Flatten())
model.add(Dense(units = 4096,activation = "relu"))
model.add(Dense(units = 4096,activation = "relu"))
model.add(Dense(units = 2, activation = "softmax"))

/home/vscode/.local/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-08-20 18:25:21.667820: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [ ]:
# definir el 

from tensorflow.keras.losses import SparseCategoricalCrossentropy

model.compile(optimizer = "adam", loss = SparseCategoricalCrossentropy(from_logits = True), metrics = ["accuracy"])

In [ ]:
#entrenamos el modelo
model.fit(X_train, y_train, epochs = 150, batch_size = 10)


In [22]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
